In [44]:
import spacy
import json
import numpy as np
from numpy.linalg import norm
from sklearn.metrics.pairwise import cosine_similarity
from spacy.training.example import Example

nlp = spacy.load("en_core_web_sm")

In [45]:
 def load_mappings(dataset):
    f = open(dataset, 'r',encoding='utf-8')
    model= dict({})
    for line in f:
        tem=line.split()
        vtem=[]
        for i in range(1,len(tem)):
            vtem.append( float(tem[i].strip()))
        vec1=np.array(vtem)
        model.update({tem[0].strip():vec1})
    return model

In [20]:
my_text = "i want to buy a blue and long sleeve shirt"

In [21]:
doc = nlp(my_text)

In [39]:
for token in doc:
    print(token.is_stop)
    print(f'token: {token}, day la .dep_: {token.dep_} và pos_: {token.pos_}')
    print(list(token.subtree), '--', token.dep_)
    if ("dobj" in token.dep_):
        subtree = list(token.subtree)
        start = subtree[0].i
        end = subtree[-1].i + 1
        extract = doc[start:end]
        product = token
        print(f'result: {doc[start:end]}')

True
token: i, day la .dep_: nsubj và pos_: PRON
[i] -- nsubj
False
token: want, day la .dep_: ROOT và pos_: VERB
[i, want, to, buy, a, blue, and, long, sleeve, shirt] -- ROOT
True
token: to, day la .dep_: aux và pos_: PART
[to] -- aux
False
token: buy, day la .dep_: xcomp và pos_: VERB
[to, buy, a, blue, and, long, sleeve, shirt] -- xcomp
True
token: a, day la .dep_: det và pos_: DET
[a] -- det
False
token: blue, day la .dep_: amod và pos_: ADJ
[blue, and, long] -- amod
True
token: and, day la .dep_: cc và pos_: CCONJ
[and] -- cc
False
token: long, day la .dep_: conj và pos_: ADJ
[long] -- conj
False
token: sleeve, day la .dep_: compound và pos_: NOUN
[blue, and, long, sleeve] -- compound
False
token: shirt, day la .dep_: dobj và pos_: NOUN
[a, blue, and, long, sleeve, shirt] -- dobj
result: a blue and long sleeve shirt


In [57]:
type(token)

spacy.tokens.token.Token

In [46]:
#load w2v
load_model = load_mappings("glove.6B.300d.txt")

In [122]:
# get_product_category 
def get_product_category(product, my_list):
    if product in load_model:
        embeddings = []
        v1 = load_model[product.strip()]
        embeddings.append(v1)
        for category in my_list:
            embeddings.append(load_model[category.strip()])
        cos_sim = cosine_similarity( [embeddings[0]], embeddings[1:])
        return my_list[np.argmax(cos_sim)]
    return 0

In [123]:
category_list = ["phone", "laptop","computer"]

In [125]:
get_product_category(token.text, category_list)

'phone'

In [170]:
# Sau khi có category thì đi vào file json để lấy ra các danh mục của item attributes
with open('data/category-list.json') as f:
    data = json.load(f)

# Lấy ra tất cả các category
phones = [item for item in data if item['category'] == get_product_category(token.text, category_list)]
for item in phones:
    attributes = item['item attributes']
type(attributes)

list_attributes = list(attributes.keys())
list_attributes

['COLOR', 'RAM', 'PRICE']